In [64]:
import json
import numpy as np
import pandas as pd

In [10]:
# def to_one_hot(numbers : list):
#     one_hot = np.zeros(7)
#     for num in numbers:
#         one_hot[num-1] += 1
#     return one_hot

In [50]:
# def parse_answer_text(text: str):
#     text = text.replace(' ', '')
#     text = text.replace('\'', '"') # Changed single quotes to double quotes around keys
#     text = text.replace("},{", ',')
#     data = json.loads(text)
#     for character in data[0]:
#         data[0][character] = json.loads(data[0][character])
#     return data

In [65]:
from sklearn.metrics import f1_score

def evaluate(predictions : pd.DataFrame|str, ground_truth : pd.DataFrame|str):
    def to_one_hot(numbers : list):
        one_hot = np.zeros(7)
        for num in numbers:
            one_hot[num-1] += 1
        return one_hot

    def parse_answer_text(text: str):
        text = text.replace(' ', '')
        text = text.replace('\'', '"') # Changed single quotes to double quotes around keys
        text = text.replace("},{", ',')
        data = json.loads(text)
        for character in data[0]:
            data[0][character] = json.loads(data[0][character])
        return data

    if isinstance(predictions, pd.DataFrame):
        pass
    elif isinstance(predictions, str):
        predictions = pd.read_csv(predictions)
    else:
        raise ValueError(
            "predictions must be a Pandas.DataFrame object, "
            "or a string representing path to a .csv file."
        )

    if isinstance(ground_truth, pd.DataFrame):
        pass
    elif isinstance(ground_truth, str):
        ground_truth = pd.read_csv(ground_truth)
    else:
        raise ValueError(
            "ground_truth must be a Pandas.DataFrame object, "
            "or a string representing path to a .csv file."
        )

    if "answers" not in predictions.columns:
        raise ValueError("predictions must have a column named \"answers\".")

    if "answers" not in ground_truth.columns:
        raise ValueError("ground_truth must have a column named \"answers\".")

    if len(predictions) != len(ground_truth):
        raise ValueError(
            "predictions and ground_truth must have the same number of rows."
        )

    f1 = 0
    num_examples = len(predictions)
    num_characters = 0
    for i in range(num_examples):
        # print(type(predictions.iloc[i]["answers"]))
        predicted_answer = parse_answer_text(predictions.iloc[i]["answers"])[0]
        true_answer = parse_answer_text(ground_truth.iloc[i]["answers"])[0]

        # print(predicted_answer, true_answer)

        num_characters_per_row = 0
        f1_per_row = 0
        for character in true_answer:
            num_characters += 1
            num_characters_per_row += 1
            try:
                predicted_classes = predicted_answer[character]
                true_classes = true_answer[character]
                predicted_one_hot = to_one_hot(predicted_classes)
                true_one_hot = to_one_hot(true_classes)
                true_positive = np.sum(predicted_one_hot * true_one_hot)
                num_predicted_positive = np.sum(predicted_one_hot)
                num_true_positive = np.sum(true_one_hot)
                f1_per_row += (2 * true_positive) / (num_predicted_positive + num_true_positive)
            except KeyError:
                f1_per_row += 0

        f1 += f1_per_row / num_characters_per_row
    f1 = f1 / num_examples
    return float(f1)

In [66]:
evaluate("train.csv", "train.csv")

1.0

In [67]:
pred = pd.read_csv("predictions.csv")[["id","story","true_answer"]]
pred = pred.rename(columns = {"true_answer":"answers"})
pred

id                                              story  \
0    DPkKiCSEs5  รอยเลื่อนสะกา ทำให้เกิดแผ่นดินไหว คร่าชีวิตเกศ...   
1    CD04c4pHIz                    กฤษฏิวุฒิสำลักหมากฝรั่งติดคอตาย   
2    TCJsISFmKl  จตุการประมาททำให้เกิดไฟไหม้ห้องพักโรงแรมจนมีผู...   
3    3PjeEYqvNS  จรินทร์เลิศขับรถสวนเข้ามาในสนามแข่งโดยไม่มีใบอ...   
4    qkGF7v4Dif  กานต์พนิตใช้เชือกเส้นใหญ่ที่เตรียมไว้ล่วงหน้า ...   
..          ...                                                ...   
355  ZvUhEREEnL  จูเง็กบุกรุกเข้าห้องประชุมและขโมยแฟ้มสำคัญออกไ...   
356  Xts0wYz8z0  คีนาทิ้งถุงขยะใหญ่กลางทางเท้า ทำให้เจียน้ำสะดุ...   
357  JM1e0SDEuP  กีตาพันธ์หลอกให้คนซื้อคอร์สเรียนภาษาอังกฤษราคา...   
358  TVZcaqGNyt  กาญณัฏฐ์ใช้ไม้เบสบอลตีศีรษะกันกนกขณะเธอหันหลัง...   
359  4qN1W7qHuB  จารุภิญญ์สร้างโปรไฟล์ปลอมว่าเป็นนายหน้าอสังหาร...   

                                            answers  
0                               [{'เกศวสา': '[1]'}]  
1                            [{'กฤษฏิวุฒิ': '[1]'}]  
2     [{'จตุการ': '[3,7]'}, {'เจษฎากรท์': '[2,4]'}]  
3                        [{'จรินทร์เลิศ': '[3,4]'}]  
4    [{'กานต์พนิต': '[5,7]'}, {'จิณห์มิกา': '[1]'}]  
..                                              ...  
355        [{'จูเง็ก': '[2,4]'}, {'กุ้ยกี': '[6]'}]  
356           [{'คีนา': '[3]'}, {'เจียน้ำ': '[1]'}]  
357                          [{'กีตาพันธ์': '[6]'}]  
358      [{'กาญณัฏฐ์': '[5,7]'}, {'กันกนก': '[1]'}]  
359                          [{'จารุภิญญ์': '[6]'}]  

[360 rows x 3 columns]

In [68]:
true = pd.read_csv("predictions.csv")[["id","story","predicted_answer"]]
true = true.rename(columns = {"predicted_answer":"answers"})
true

id                                              story  \
0    DPkKiCSEs5  รอยเลื่อนสะกา ทำให้เกิดแผ่นดินไหว คร่าชีวิตเกศ...   
1    CD04c4pHIz                    กฤษฏิวุฒิสำลักหมากฝรั่งติดคอตาย   
2    TCJsISFmKl  จตุการประมาททำให้เกิดไฟไหม้ห้องพักโรงแรมจนมีผู...   
3    3PjeEYqvNS  จรินทร์เลิศขับรถสวนเข้ามาในสนามแข่งโดยไม่มีใบอ...   
4    qkGF7v4Dif  กานต์พนิตใช้เชือกเส้นใหญ่ที่เตรียมไว้ล่วงหน้า ...   
..          ...                                                ...   
355  ZvUhEREEnL  จูเง็กบุกรุกเข้าห้องประชุมและขโมยแฟ้มสำคัญออกไ...   
356  Xts0wYz8z0  คีนาทิ้งถุงขยะใหญ่กลางทางเท้า ทำให้เจียน้ำสะดุ...   
357  JM1e0SDEuP  กีตาพันธ์หลอกให้คนซื้อคอร์สเรียนภาษาอังกฤษราคา...   
358  TVZcaqGNyt  กาญณัฏฐ์ใช้ไม้เบสบอลตีศีรษะกันกนกขณะเธอหันหลัง...   
359  4qN1W7qHuB  จารุภิญญ์สร้างโปรไฟล์ปลอมว่าเป็นนายหน้าอสังหาร...   

                                               answers  
0                     [{'เกศ': '[1]'}, {'วสา': '[1]'}]  
1                  [{'วุฒิ': '[1]'}, {'กฤษฏิ': '[1]'}]  
2    [{'จตุ': '[2]'}, {'การ': '[2]'}, {'กรท์': '[2]...  
3    [{'เลิศ': '[3]'}, {'นทร์': '[3]'}, {'ริ': '[3]'}]  
4    [{'กา': '[1,5,7]'}, {'จิณห์': '[1,5,7]'}, {'มิ...  
..                                                 ...  
355  [{'กี': '[2,4,6]'}, {'กุ้ย': '[2,4,6]'}, {'เง็...  
356  [{'เจีย': '[3]'}, {'น้ำ': '[3]'}, {'นา': '[3]'...  
357   [{'กี': '[6]'}, {'พันธ์': '[6]'}, {'ตา': '[6]'}]  
358  [{'กัน': '[5,7]'}, {'กา': '[5]'}, {'กนก': '[5,...  
359                [{'ภิญญ์': '[6]'}, {'จารุ': '[6]'}]  

[360 rows x 3 columns]

In [69]:
evaluate(pred, true)

0.05309413580246914